In [1]:
import notebookimport

In [2]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")
sp = __import__("Charts - Stage Progress")


importing Jupyter notebook from Charts - Stage Results.ipynb
importing Jupyter notebook from Charts - Split Sector Delta.ipynb
importing Jupyter notebook from Charts - Stage Progress.ipynb


In [3]:
import os
import sqlite3
import pandas as pd

#dbname='wrc18.db'
dbname='mexico18.db'
conn = sqlite3.connect(dbname)

In [4]:
rally='Mexico'

In [5]:
'''year=2018
rc='RC2'
ss='SS4'


typ='stage_times_stage' #stage_times_stage stage_times_overall
typ='stage_times_overall'''

"year=2018\nrc='RC2'\nss='SS4'\n\n\ntyp='stage_times_stage' #stage_times_stage stage_times_overall\ntyp='stage_times_overall"

In [6]:
stageresults = sr.dbGetRallyStages(conn, rally)
stagerank_overall = sr.getEnrichedStageRank(conn,rally, typ='overall')

stagerank_stage = sr.getEnrichedStageRank(conn,rally, typ='stage')

In [7]:
basepath = 'report'
imgdir = 'images'
imgdirfull = '{}/{}'.format(basepath, imgdir)


In [7]:
sections = stageresults.groupby('section')

#Generate list from group: stageresults.groupby('section')['code'].apply(list)

#Generate group then iterate through groups
for key in sections.groups.keys():
    print(key, sections.get_group(key)['code'].tolist())


Section 1 ['SS1']
Section 2 ['SS2', 'SS3', 'SS4', 'SS5']
Section 3 ['SS6', 'SS7', 'SS8', 'SS9', 'SS10']
Section 4 ['SS11', 'SS13', 'SS12']
Section 5 ['SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19']
Section 6 ['SS20', 'SS21']
Section 7 ['SS22']


In [9]:
sectionREADME_base = '''\n### {section} Report
'''

## Section Reports

In [10]:
%%capture
if not os.path.exists(imgdirfull):
    os.makedirs(imgdirfull)

def sectionRankingChart(data, fn,sectionREADME, key, deltalabels=True, elaborate=True):
    data = sr.stageOverallEnrichers(data)
    ss = data[data['code'].isin(sectionStages)].copy()
    ss = sr.stageLegEnrichers(ss)
    fn='{}/{}'.format(imgdirfull, fn_core)
    fig, ax = sr.plotStageProgressionChart( ss, linecolor='lightgrey' , deltalabels=deltalabels, elaborate=elaborate,
                                        progress=True, stageAnnotate=False, filename=fn, title = title); 
    if fig is not None:
        fn='{}/{}'.format(imgdir, fn_core)
        sectionREADME = '''{s}{key}\n\n![]({img})'''.format(s=sectionREADME, key=key,img=fn)
    return sectionREADME

for key in sections.groups.keys():
    #path = '{}/{}'.format(basepath,key)
    #if not os.path.exists(path):
    #    os.makedirs(path)
    sectionStages = sections.get_group(key)['code'].tolist()
    
    sectionREADME = sectionREADME_base.format(section=key)
        
    #Stage rank chart
    fn_core='spchart_stage_{}.png'.format( key)
    title = '{} - Stage Ranking'.format(key)
    print(key)
    sectionREADME = sectionRankingChart(stagerank_stage, title, sectionREADME, key, deltalabels=False, elaborate=False)
    
    #Overall rally ranking chart
    fn_core='spchart_overall_{}.png'.format( key)
    title = '{} - Overall Rally Ranking Evolution'.format(key)
    sectionREADME = sectionRankingChart(stagerank_overall, title, sectionREADME, key, deltalabels=True, elaborate=True )
    
    with open('{bp}/{key}_report.md'.format(bp=basepath,key=key), 'a') as out_file:
        out_file.write(sectionREADME)


In [11]:
key

'Section 7'

### Stage progression chart

In [8]:
%%capture
fn='{}/spchart_full.png'.format(imgdirfull)
fig, ax = sr.plotStageProgressionChart( stagerank_overall, linecolor='lightgrey' , deltalabels=True, 
                                        progress=True, stageAnnotate=False, filename=fn);

README='''\n![]({})'''.format('{}/spchart_full.png'.format(imgdir))
with open('{}/README.md'.format(basepath), 'a') as out_file:
    out_file.write(README)

In [9]:
!ls report

README.md           Section 2_report.md Section 5_report.md images
SS1_report.md       Section 3_report.md Section 6_report.md
Section 1_report.md Section 4_report.md Section 7_report.md


## Stage Reports

In [8]:
overallleader = stagerank_overall[stagerank_overall['position']==1].set_index('code')[['driver.code']].to_dict(orient='dict')['driver.code']
overallleader

{'SS1': 'NEU',
 'SS10': 'SOR',
 'SS11': 'LOE',
 'SS12': 'LOE',
 'SS13': 'LOE',
 'SS14': 'OGI',
 'SS15': 'OGI',
 'SS16': 'OGI',
 'SS17': 'OGI',
 'SS18': 'OGI',
 'SS19': 'OGI',
 'SS2': 'MEE',
 'SS20': 'OGI',
 'SS21': 'OGI',
 'SS22': 'OGI',
 'SS3': 'SOR',
 'SS4': 'SOR',
 'SS5': 'SOR',
 'SS6': 'SOR',
 'SS7': 'SOR',
 'SS8': 'SOR',
 'SS9': 'SOR'}

In [9]:
rc='RC1'
year=2018
ss='SS4'

In [10]:
import matplotlib.pyplot as plt
import unidecode

links=[]
for ss in stageresults['code']:
    print(ss)
    #if ss=='SS1': continue
    currprevstagerank = sp.getCurrPrevStageRank(conn, rally, rc, ss)
    splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
    with open('{}/{}_report.md'.format(basepath, ss), 'w') as f:
        #for dc in currprevstagerank['driver.code'].unique():
        dc = overallleader[ss]
        fig, ax = sp.rebaseStageProgressBar(currprevstagerank,dc, splitdurations=splitdurations,
                                            neg='lightgreen',pos='pink',
                                 title='Stage Progress Chart - {rally}, {year} - SS{ss}'.format(year=year,
                                                                                                rally=rally,
                                                                                                ss=str(ss).replace('SS','')));
        img='{}/stage_report_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
        fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
        plt.close(fig)  
        f.write('# Stage Progress Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                              dc=dc,
                                                                              ss=str(ss).replace('SS','')))
        f.write('![]({})\n'.format(img))

SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS13
SS12
SS14
SS15
SS16
SS17
SS18
SS19
SS20
SS21
SS22


In [11]:
ssd.dbGetSplits(conn,rally,'SS10',rc='RC1').empty
#ssd.getSplitDurationsFromSplits(conn,rally,'SS9',rc)

True